In [1]:
from parsing import *
from database_functions import *
from ml_functions import *

In [ ]:
#patients, appointments, weather = parsing(data_raw_fname = '/Users/jenniferpolson/Documents/School/2017-F/BE 223A/Final Project/Functionalize/be223a_dataset_full.csv', 
#             encoding = "ISO-8859-1", 
#             dtformat = '%Y-%m-%d %H:%M:%S', 
#            exam_id = 'examId', pt_id = 'patient_deid', age = 'AgeAtExam', gender = 'Gender')

In [2]:
data_raw_fname = '/Users/jenniferpolson/Documents/School/2017-F/BE 223A/Midterm Project/be223a_dataset.csv'
encoding=None
dtformat='%m/%d/%y %H:%M'
patients, appointments, weather = parsing(data_raw_fname, encoding, dtformat, 
                                          exam_id = 'Exam ID', pt_id = 'Patient ID', age = 'Age', gender = 'Gender')

Reading /Users/jenniferpolson/Documents/School/2017-F/BE 223A/Midterm Project/be223a_dataset.csv
Processed in 3.661 seconds.


## This is where the DB goes

In [3]:
def generate_string(tablename, df, primarykey, foreignkey):
    
    """ Function to generate the SQL command as a string. This function can be used when only primary keys have to be created and there are no foreign keys associated.
    
    Arguments:
    tablename (char): Name of the table to be created in the database
    df (dataframe): Parsed dataframe output from which field names for the database will be extracted from the column names
    primarykey (char): Primary unique key associated with the table, this should be one of the columnnames from the dataframe
       
    
    Returns:
    An SQL command as a string which can be executed to create tables
    
    """
    
    columnnames=list(df.columns.values)
    columnnames=[item for item in columnnames if item not in foreignkey]
    
    sql_string="CREATE TABLE IF NOT EXISTS"+ " "+ tablename + "("
    
 
        
    for i in range(len(columnnames)):

        if i==len(columnnames)-1:
            col=columnnames[i]
            col_type=df[columnnames[i]].dtype

            if col==primarykey:
                coltype=type_col(col_type)
                sql_string = sql_string + col + " " + coltype + " " + "PRIMARY KEY" + " " + ")" + ";" 
            else:
                coltype=type_col(col_type)
                sql_string= sql_string + col + " " + coltype + " " + ")" + ";"

        else:

            col=columnnames[i]
            col_type=df[columnnames[i]].dtype

            if col==primarykey:
                coltype=type_col(col_type)
                sql_string = sql_string + col + " " + coltype + " " + "PRIMARY KEY" + ","

            else:
                coltype=type_col(col_type)
                sql_string= sql_string + col + " " + coltype + ","

    return (sql_string)

def create_key(database, foreignstring):
    connection = sqlite3.connect(database)
    cursor=connection.cursor()
    cursor.execute(foreignstring)
    connection.commit()
    connection.close()

In [4]:
#Input Parameters
dbname = "223ADB3.db"
patient_key = "Patient_ID"
appointment_key = "Exam_ID"
weather_key = "Dayofyear"

In [5]:
connection = new_connection(dbname)

In [6]:
#Generating SQL Strings
patients_string=generate_string("patients", patients , patient_key, [])
appointments_string=generate_string("appointments", appointments, appointment_key, [patient_key, weather_key])
weather_string=generate_string("weather", weather, "Dayofyear", [])

In [7]:
# Creating tables

create_table(dbname, patients_string)
create_table(dbname, appointments_string)
create_table(dbname, weather_string)

OperationalError: near "Group": syntax error

In [8]:
# Creating foreign keys

fkey1=foreignkey("appointments", patient_key, "patients", patient_key)
fkey2=foreignkey("appointments", weather_key, "weather", weather_key)


# Creating foreign key columns
#create_key(dbname, fkey1)
#create_key(dbname, fkey2)

In [9]:
# Push dataframe into database

push_dataframe(patients,dbname, "patients")
push_dataframe(appointments, dbname, "appointments")
push_dataframe(weather,dbname, "weather")

In [10]:
#other function: pull from DB, rearrange to data output
patients = query_data(dbname, "SELECT * from patients") 
appointments = query_data(dbname, "SELECT * from appointments")
weather = query_data(dbname, "SELECT * from weather")

,index,Patient ID,Age,Gender
0,0,0000219e4b37d2504fb6b8c28e24a2d4,85,F
1,1,0000a7ddc7607d1d8ee5c47dd437034e,31,F
2,2,0000e1b68421441f1bb8e697ecdad119,61,F
3,3,00028fae880618d73d2dcfa3e5bc670b,68,F
4,5,0002aac1c9addac1d4ee6902d6ec5404,64,F
5,6,0004fb840f61b62300e85fff9d5d5507,21,F
6,7,0005bf180d7043a1959152398505d12b,39,M
7,8,00069bac527f81cc1fd8f6a1d6ab8ec0,69,F
8,9,0006ff704be72d3514db32fcbbcb5885,33,F
9,10,00071ad911d36fa0706bb2c70b80c799,64,M


## Start here for merging

In [11]:
df = create_df(appointments, patients, weather, 'Patient_ID', 'Dayofyear').drop('index', axis = 1)

ValueError: list.remove(x): x not in list

In [ ]:
#run ML
results, test, prob, groups, evalstats = run_model(df, k = 5000, gridsearch = False)

In [ ]:
#create new table
results_string = generate_string ('results', results, 'Patient_ID', []) #patient ID
create_table(dbname, results_string)

#push the data to the table
push_dataframe(results,dbname, 'results')

In [ ]:
#circular - to make sure it actually worke
results = query_data(dbname, "SELECT * from results") 